In [1]:
#!/usr/bin/python3
import re
import subprocess
import os
os.getcwd()
os.chdir("data")


In [2]:
# Run BLASTP
subprocess.call(['makeblastdb', '-in', 'Rossellomorea_vietnamemsis_RAR_WA_32.faa', '-dbtype', 'prot', '-parse_seqids', '-out', 'Rossellomorea_vietnamemsis_RAR_WA_32'], stdout=subprocess.PIPE)


0

In [3]:
subprocess.call(["blastp", "-query", 'MCCB_382.faa', "-subject", 'Rossellomorea_vietnamemsis_RAR_WA_32.faa', "-out", "MCCB_382_vs_Rossellomorea_vietnamemsis_RAR_WA_32.tab", "-evalue", "1e-5", "-outfmt", "6 std qlen",'-max_target_seqs', '1'])



0

In [4]:
import pandas as pd
query_results = pd.read_csv('MCCB_382_vs_Rossellomorea_vietnamemsis_RAR_WA_32.tab', sep="\t", header=None)
print(query_results)

                  0               1       2    3   4   5   6    7   8    9   \
0     KBBLJFAO_00001  CNFHIFKK_04266  96.732  153   5   0   1  153   1  153   
1     KBBLJFAO_00002  CNFHIFKK_04265  92.857  182  13   0   1  182   1  182   
2     KBBLJFAO_00003  CNFHIFKK_04264  96.389  360  13   0   1  360   1  360   
3     KBBLJFAO_00004  CNFHIFKK_04263  99.632  815   3   0   1  815   1  815   
4     KBBLJFAO_00005  CNFHIFKK_04262  99.563  458   2   0   1  458   1  458   
...              ...             ...     ...  ...  ..  ..  ..  ...  ..  ...   
4179  KBBLJFAO_04806  CNFHIFKK_04360  99.317  293   2   0   1  293   1  293   
4180  KBBLJFAO_04807  CNFHIFKK_04361  86.918  451  59   0   1  451   1  451   
4181  KBBLJFAO_04808  CNFHIFKK_04362  99.795  487   1   0   1  487   1  487   
4182  KBBLJFAO_04809  CNFHIFKK_04363  86.129  310  43   0   1  310   1  310   
4183  KBBLJFAO_04810  CNFHIFKK_04364  93.671  158  10   0   1  158   1  158   

                 10      11   12  
0     7.300000e-

In [5]:
query_results= query_results.drop_duplicates(subset=[0, 1])
len(query_results[1])

4169

In [6]:
# Run BLASTP
subprocess.call(['makeblastdb', '-in', 'MCCB_382.faa', '-dbtype', 'prot', '-parse_seqids', '-out', 'MCCB_382'], stdout=subprocess.PIPE)


0

In [7]:
subprocess.call(["blastp", "-query", 'Rossellomorea_vietnamemsis_RAR_WA_32.faa', "-subject", 'MCCB_382.faa', "-out", "Rossellomorea_vietnamemsis_RAR_WA_32_vs_MCCB_382.tab", "-evalue", "1e-5", "-outfmt", "6 std qlen",'-max_target_seqs', '1'])


0

In [9]:
subject_results = pd.read_csv('Rossellomorea_vietnamemsis_RAR_WA_32_vs_MCCB_382.tab', sep="\t", header=None)


In [10]:
#Filter duplicates based on columns 0 and 1
subject_results = subject_results.drop_duplicates(subset=[0, 1])
len(subject_results[1])

4143

In [11]:
pattern = r">(\S+)"

# Processing MCCB_382.faa
with open("MCCB_382.faa", "r") as f:
    query_data = f.read()

query_matches = re.findall(pattern, query_data)
query_counts = {}
for query_id in set(query_matches):
    query_count = query_matches.count(query_id)
    query_counts[query_id] = query_count

# Processing Rossellomorea_vietnamemsis_RAR_WA_32.faa
with open("Rossellomorea_vietnamemsis_RAR_WA_32.faa", "r") as f:
    subject_data = f.read()

subject_matches = re.findall(pattern, subject_data)
subject_counts = {}
for subject_id in set(subject_matches):
    subject_count = subject_matches.count(subject_id)
    subject_counts[subject_id] = subject_count

In [12]:
print(len(query_counts))
from IPython.display import display
display(len(subject_counts))

4698


4517

In [13]:
# Create query and subject coverage columns in both dataframes
query_results['qcov'] = query_results[3] / query_results[12]
subject_results['qcov'] = subject_results[3] / subject_results[12]

# Apply your custom filtering logic
threshold_ident = 40
threshold_qcov = 0.5

# Filter based on conditions
query_conserved_protein = query_results[(query_results[2] >= threshold_ident) & (query_results[12] >= threshold_qcov)]
subject_conserved_protein = subject_results[(subject_results[2] >= threshold_ident) & (subject_results[12] >= threshold_qcov)]


In [16]:
query_conserved_protein.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,qcov
0,KBBLJFAO_00001,CNFHIFKK_04266,96.732,153,5,0,1,153,1,153,7.300000e-108,301.0,153,1.0
1,KBBLJFAO_00002,CNFHIFKK_04265,92.857,182,13,0,1,182,1,182,6.550000e-127,351.0,182,1.0
2,KBBLJFAO_00003,CNFHIFKK_04264,96.389,360,13,0,1,360,1,360,0.000000e+00,708.0,360,1.0
3,KBBLJFAO_00004,CNFHIFKK_04263,99.632,815,3,0,1,815,1,815,0.000000e+00,1639.0,815,1.0
4,KBBLJFAO_00005,CNFHIFKK_04262,99.563,458,2,0,1,458,1,458,0.000000e+00,934.0,458,1.0


In [17]:

print(len(query_conserved_protein))

print(len(subject_conserved_protein))

3945
3923


In [18]:
#Filter duplicates based on columns 0 and 1
query_conserved_protein = query_conserved_protein.drop_duplicates(subset=[0, 1])
subject_conserved_protein= subject_conserved_protein.drop_duplicates(subset=[0, 1])

print(len(query_conserved_protein))

print(len(subject_conserved_protein))


3945
3923


In [19]:
# Calculate the final percentage
total_queries = float(len(query_counts))+float(len(subject_counts))
hit_sum = float(len(query_conserved_protein[1].unique())) + float(len(subject_conserved_protein[1].unique()))
percentage = (hit_sum / total_queries) * 100

print(percentage)


83.44004340748779


In [20]:
# Calculate the final percentage
total_queries = len(query_counts)+len(subject_counts)
hit_sum = len(query_conserved_protein[1]) + len(subject_conserved_protein[1])
percentage = (hit_sum / total_queries) * 100

print(percentage)

85.38252848616386
